# Initializing Libraries

In [ ]:
import logging
import sqlite3
from typing import Annotated, List, Literal, Optional, Tuple
from typing_extensions import TypedDict

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("RAG_Student_System")

# Install required packages
logger.info("Installing required packages...")
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

import os

# Get API key - In Colab, use userdata; otherwise use environment variables

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# PART 1: DATABASE INITIALIZATION

In [2]:
class DatabaseManager:
    """Manages SQLite database operations for the RAG system"""

    def __init__(self, db_path: Optional[str] = None):
        """
        Initialize database connection

        Args:
            db_path: Path to SQLite database file. If None, uses in-memory database
        """
        self.db_path = db_path if db_path else ':memory:'
        logger.info(f"Initializing database at {self.db_path}")
        self.conn = sqlite3.connect(self.db_path , check_same_thread=False)
        self.cursor = self.conn.cursor()

    def setup_schema(self):
        """Create the students table schema"""
        logger.info("Creating students table schema")
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                first_name TEXT,
                last_name TEXT,
                age INTEGER,
                major TEXT,
                gpa REAL,
                marital_status TEXT,
                education_status TEXT
            )
        ''')
        self.conn.commit()

    def populate_sample_data(self, data: List[Tuple]):
        """
        Populate the database with sample student records

        Args:
            data: List of student record tuples
        """
        logger.info(f"Populating database with {len(data)} sample records")
        self.cursor.executemany('''
            INSERT INTO students
            (first_name, last_name, age, major, gpa, marital_status, education_status)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', data)
        self.conn.commit()

    def run_test_query(self, limit: int = 3):
        """Run a test query to verify database setup"""
        logger.info("Running test query")
        results = []
        for row in self.cursor.execute(f"SELECT * FROM students LIMIT {limit}"):
            results.append(row)
            logger.info(f"Test row: {row}")
        return results

    def execute_query(self, query: str) -> List[Tuple]:
        """
        Execute an SQL query and return results

        Args:
            query: SQL query string

        Returns:
            List of result tuples
        """
        logger.debug(f"Executing query: {query}")
        try:
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            logger.debug(f"Query returned {len(rows)} results")
            return rows
        except Exception as e:
            logger.error(f"Database query error: {e}")
            raise

# Create database instance
db_manager = DatabaseManager()

# PART 2: LANGGRAPH STATE AND TOOLS

In [3]:
# Define the state for our LangGraph
class StudentQueryState(TypedDict):
    messages: Annotated[list, add_messages]
    finished: bool

# System prompt for the student database assistant
STUDENT_DB_SYSINT = (
    "system",
    "You are a Student Database Assistant, helping users query and analyze student records. "
    "You can answer questions about students in the database, including their personal details, "
    "academic information, and statistics. You should always use the ask_students_db tool to query "
    "the database when answering questions about specific data or statistics. "
    "\n\n"
    "Be professional, concise, and accurate in your responses. If a query returns no results, "
    "explain this clearly. If a user asks a question that requires data outside your knowledge or "
    "the database, politely explain that you can only provide information contained in the student database."
    "\n\n"
    "Always verify that SQL queries are correctly formatted before executing them, and interpret the "
    "results in a human-friendly way after receiving them."
)

WELCOME_MSG = "Welcome to the Student Database Assistant. I can help you query information about students. What would you like to know?"

# Define tool for querying the database
@tool
def ask_students_db(query: str) -> str:
    """
    Execute SQL queries on the students table and return the results.
    The students table has the following columns:
    - id (INTEGER)
    - first_name (TEXT)
    - last_name (TEXT)
    - age (INTEGER)
    - major (TEXT)
    - gpa (REAL)
    - marital_status (TEXT)
    - education_status (TEXT)
    """
    logger.info(f"Executing query: {query}")
    try:
        rows = db_manager.execute_query(query)
        if not rows:
            return "No results found for this query."

        out = []
        for r in rows:
            out.append(str(r))
        result = "\n".join(out)
        return result
    except Exception as e:
        error_msg = f"Error executing query: {e}"
        logger.error(error_msg)
        return error_msg


# PART 3: LANGGRAPH NODES

In [4]:
# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# Create LLM with tools binding
tools = [ask_students_db]
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools)

def chatbot_node(state: StudentQueryState) -> StudentQueryState:
    """Node for generating responses with the LLM"""
    defaults = {"finished": False}

    if state.get("messages", []):
        logger.info("Generating LLM response")
        new_output = llm_with_tools.invoke([STUDENT_DB_SYSINT] + state["messages"])
    else:
        logger.info("Generating welcome message")
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}

def human_node(state: StudentQueryState) -> StudentQueryState:
    """Node for handling human input"""
    last_msg = state["messages"][-1]
    print("Assistant:", last_msg.content)
    user_input = input("User: ")

    # Check for exit commands
    if user_input.strip().lower() in ["q", "quit", "exit", "goodbye"]:
        logger.info("User requested to exit")
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}

# PART 4: GRAPH ROUTING FUNCTIONS

In [5]:
def route_based_on_tools(state: StudentQueryState) -> Literal["tools", "human"]:
    """Route to tools if needed, otherwise to human node"""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]
    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        logger.debug("Routing to tools node")
        return "tools"
    else:
        logger.debug("Routing to human node")
        return "human"

def maybe_exit_human_node(state: StudentQueryState) -> Literal["chatbot", "__end__"]:
    """Check if we should exit or continue the conversation"""
    if state.get("finished", False):
        logger.info("Conversation finished, ending graph")
        return END
    else:
        logger.debug("Continuing conversation")
        return "chatbot"

# PART 5: BUILD AND RUN THE GRAPH

In [6]:

def build_graph():
    """Build the LangGraph for the student database assistant"""
    logger.info("Building LangGraph")

    graph_builder = StateGraph(StudentQueryState)

    # Add nodes
    graph_builder.add_node("chatbot", chatbot_node)
    graph_builder.add_node("human", human_node)
    graph_builder.add_node("tools", tool_node)

    # Add edges
    graph_builder.add_edge(START, "chatbot")
    graph_builder.add_conditional_edges("chatbot", route_based_on_tools)
    graph_builder.add_conditional_edges("human", maybe_exit_human_node)
    graph_builder.add_edge("tools", "chatbot")

    # Compile the graph
    return graph_builder.compile()

def setup_database():
    """Setup and populate the student database"""
    logger.info("Setting up database")

    # Sample student data
    sample_data = [
        ("Ali", "Rezaei", 20, "Computer Engineering", 16.2, "Single", "Undergraduate"),
        ("Zahra", "Kamali", 24, "Mathematics", 17.4, "Married", "Master"),
        ("Sina", "Nazari", 22, "Physics", 18.1, "Single", "Undergraduate"),
        ("Maryam", "Moradi", 26, "Computer Engineering", 15.8, "Single", "Master"),
        ("Nima", "Ahmadi", 23, "Computer Engineering", 14.7, "Married", "Undergraduate"),
        ("Fatemeh", "Azizi", 29, "Chemistry", 17.9, "Single", "PhD"),
        ("Reza", "Mohebbi", 21, "Mathematics", 13.4, "Single", "Undergraduate"),
        ("Sara", "Pakdel", 27, "Chemistry", 19.1, "Married", "PhD"),
        ("Ahmad", "Ghorbani", 28, "Physics", 16.5, "Single", "Master"),
        ("Nazanin", "Hosseini", 30, "Industrial Engineering", 14.2, "Married", "PhD"),
        ("Amir", "Shirazi", 25, "Mathematics", 16.0, "Single", "Master"),
        ("Parisa", "Fattahi", 24, "Industrial Engineering", 15.2, "Single", "Undergraduate"),
        ("Hamed", "Alavi", 19, "Computer Engineering", 12.8, "Single", "Undergraduate"),
        ("Shima", "Kavian", 31, "Physics", 17.3, "Married", "PhD"),
        ("Yasaman", "Rahmani", 22, "Industrial Engineering", 14.4, "Single", "Undergraduate"),
        ("Ali", "Bagheri", 32, "Chemistry", 16.9, "Married", "PhD"),
        ("Elham", "Najafi", 28, "Mathematics", 18.2, "Married", "Master"),
        ("Reyhane", "Mohseni", 21, "Computer Engineering", 16.7, "Single", "Undergraduate"),
        ("Hamid", "Zakeri", 25, "Physics", 15.3, "Single", "Master"),
        ("Mona", "Soltani", 27, "Chemistry", 16.1, "Single", "Master"),
    ]

    db_manager.setup_schema()
    db_manager.populate_sample_data(sample_data)
    db_manager.run_test_query()

def run_interactive_session(graph, initial_message=None):
    """Run an interactive session with the graph"""
    logger.info("Starting interactive session")

    # Set up initial state
    if initial_message:
        state = {"messages": [("user", initial_message)]}
    else:
        state = {"messages": []}

    # Run the graph
    try:
        config = {"recursion_limit": 50}
        graph.invoke(state, config=config)
        logger.info("Interactive session completed")
    except Exception as e:
        logger.error(f"Error in interactive session: {e}")
        raise


In [ ]:
def main():
    """Main function to run the student database RAG application"""
    try:
        # Set up the database
        setup_database()

        # Build the graph
        graph = build_graph()

        # Optional: Visualize the graph if in a notebook environment
        try:
            from IPython.display import Image, display
            display(Image(graph.get_graph().draw_mermaid_png()))
        except ImportError:
            logger.info("IPython not available, skipping graph visualization")

        # Run interactive session
        initial_question = "How many students do we have?"
        run_interactive_session(graph, initial_question)

    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

if __name__ == "__main__":
    main()